# Metro transit information 

Using the metro API, this notebook gets data on travel times to Hattie Mae White on board meeting days. 

In [1]:
import numpy as np
import pandas as pd
import requests
import time
import dateutil

direc = "//Users//afan//Desktop//Misc//HMW_Transit//"

## Testing things

It's been a while since I've worked with an API, so let's just make sure some things are right

In [2]:
# Let's get all the information on the one route I ride the most often
response = requests.get("https://api.ridemetro.org/data/Routes?$filter=RouteName eq '004'&subscription-key=4e7902d1a2ae42df978fe20387049854")
print(response.status_code)

200


In [3]:
response.json()

{'@odata.context': 'http://api.transitiq.com/ODataService.svc/$metadata#Routes',
 'value': [{'RouteId': 'Ho414_4620_41894',
   'AgencyAbbreviation': 'RideMetro',
   'FinalStop0Id': None,
   'FinalStop1Id': None,
   'RouteName': '004',
   'LongName': 'Beechnut',
   'RouteType': 'Bus',
   'DataSource': 'Transit_5'}]}

In [5]:
# Now let's get all the information we can about this route. This is literally all the stops! 
response = requests.get("https://api.ridemetro.org/data/Routes('Ho414_4620_41894')?subscription-key=4e7902d1a2ae42df978fe20387049854&$expand=Stops")
print(response.status_code)

200


In [6]:
response.json()

{'@odata.context': 'http://api.transitiq.com/ODataService.svc/$metadata#Routes(Stops())',
 'value': [{'RouteId': 'Ho414_4620_41894',
   'AgencyAbbreviation': 'RideMetro',
   'FinalStop0Id': None,
   'FinalStop1Id': None,
   'RouteName': '004',
   'LongName': 'Beechnut',
   'RouteType': 'Bus',
   'DataSource': 'Transit_5',
   'Stops': [{'Id': 'Ho414_4620_94_D_Ho414_4620_41894_0',
     'StopId': 'Ho414_4620_94_D',
     'DirectionId': 'Ho414_4620_41894_0',
     'RouteId': 'Ho414_4620_41894',
     'AgencyId': 'Ho414',
     'Name': 'MISSION BEND TRANSIT CENTER BAY D',
     'StopCode': '94',
     'Type': 'Bus',
     'Lat': 29.711304,
     'Lon': -95.62981,
     'Ordinal': 1,
     'OrdinalOnDirection': 1},
    {'Id': 'Ho414_4620_12323_Ho414_4620_41894_0',
     'StopId': 'Ho414_4620_12323',
     'DirectionId': 'Ho414_4620_41894_0',
     'RouteId': 'Ho414_4620_41894',
     'AgencyId': 'Ho414',
     'Name': 'Howell Sugarland Rd @ Rio Bonito Rd',
     'StopCode': '12323',
     'Type': 'Bus',
    

In [7]:
# Let's get all the vehicles
response = requests.get("https://api.ridemetro.org/data/Vehicles?subscription-key=4e7902d1a2ae42df978fe20387049854")
print(response.status_code)


200


In [8]:
response.json()

{'@odata.context': 'http://api.transitiq.com/ODataService.svc/$metadata#Vehicles',
 'value': [{'VehicleId': '1908',
   'AgencyId': 'Ho414',
   'RouteId': 'Ho414_4620_41949',
   'RouteName': '084',
   'TripId': 'Ho414_4620_9274721',
   'DirectionName': '084, NORTHBOUND',
   'DestinationName': 'NORTHWEST TC',
   'TripStartTimeLocal': '2022-01-04T22:00:00Z',
   'TripEndTimeLocal': '2022-01-04T22:51:00Z',
   'TripStartTimeUTC': '2022-01-05T04:00:00Z',
   'TripEndTimeUTC': '2022-01-05T04:51:00Z',
   'Delayseconds': 639,
   'VehicleReportTime': '2022-01-05T04:20:29Z',
   'IsMonitored': True,
   'Latitude': 29.6926803588867,
   'Longitude': -95.4132690429688,
   'Block': None},
  {'VehicleId': '1943',
   'AgencyId': 'Ho414',
   'RouteId': 'Ho414_4620_41901',
   'RouteName': '011',
   'TripId': 'Ho414_4620_9260734',
   'DirectionName': '011, NORTHBOUND',
   'DestinationName': 'GELLHORN & 610',
   'TripStartTimeLocal': '2022-01-04T21:30:00Z',
   'TripEndTimeLocal': '2022-01-04T22:51:00Z',
   'T

## Exploring bus stops around Hattie Mae White

Let's start with how accessible Hattie Mae White is by public transit: Let's see what stops are in the area

In [9]:
# Hattie Mae White coordinatess

HMW_lat = 29.802759908899148
HMW_long = -95.45410037006431

key = "4e7902d1a2ae42df978fe20387049854"

In [10]:
HMW_stops = requests.get(f"https://api.ridemetro.org/data/GeoAreas('{HMW_lat}|{HMW_long}|0.3')/Stops?subscription-key={key}")
print(HMW_stops.status_code)

200


In [11]:
HMW_stops.json()

{'@odata.context': 'http://api.transitiq.com/ODataService.svc/$metadata#Stops',
 'value': [{'StopId': 'Ho414_4620_8412',
   'AgencyId': 'Ho414',
   'Name': 'W 18Th St @ Mangum Rd',
   'StopCode': '8412',
   'Type': None,
   'DataSource': '',
   'Lat': 29.801395,
   'Lon': -95.455853,
   'DistanceFromCenter': 0.19903016518379862},
  {'StopId': 'Ho414_4620_8413',
   'AgencyId': 'Ho414',
   'Name': 'W 18Th St @ Northwest Freeway',
   'StopCode': '8413',
   'Type': None,
   'DataSource': '',
   'Lat': 29.801401,
   'Lon': -95.45297,
   'DistanceFromCenter': 0.16143556708060636},
  {'StopId': 'Ho414_4620_12077',
   'AgencyId': 'Ho414',
   'Name': 'W 18Th St @ Northwest Fwy',
   'StopCode': '12077',
   'Type': None,
   'DataSource': '',
   'Lat': 29.80163,
   'Lon': -95.454342,
   'DistanceFromCenter': 0.09251021044187172},
  {'StopId': 'Ho414_4620_11053',
   'AgencyId': 'Ho414',
   'Name': 'Hempstead Rd @ Post Oak Rd',
   'StopCode': '11053',
   'Type': None,
   'DataSource': '',
   'Lat': 